# SpectraMind V50 — Diagnostics & Explainability

Run the dashboard and display the HTML inline (if present).

In [ ]:
import sys, subprocess, shlex
from pathlib import Path

def choose_cli() -> str:
    def ok(cmd: str) -> bool:
        try:
            subprocess.run(cmd + " --help", shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return True
        except Exception:
            return False
    for cmd in ("poetry run spectramind", "spectramind", f"{shlex.quote(sys.executable)} -m spectramind"):
        if ok(cmd):
            return cmd
    return "spectramind"

CLI = choose_cli()
OUT = Path("outputs/diagnostics")
OUT.mkdir(parents=True, exist_ok=True)
CLI, str(OUT)

## Generate dashboard (light)
Skips heavy projections by default; falls back to full if supported.

In [ ]:
cmd = CLI + f" diagnose dashboard --no-umap --no-tsne --outdir {shlex.quote(str(OUT))}"
rc = subprocess.call(cmd, shell=True)
if rc != 0:
    subprocess.call(CLI + f" diagnose dashboard --outdir {shlex.quote(str(OUT))}", shell=True)
report = OUT / "report.html"
report.exists(), str(report)

## Render inline
If the HTML report exists, show it right here as an iframe.

In [ ]:
from IPython.display import IFrame, display, HTML
report = Path("outputs/diagnostics/report.html")
if report.exists():
    display(IFrame(src=str(report), width="100%", height="640px"))
else:
    display(HTML("<p style='color:#c00'>Report not found. Run the cell above to generate diagnostics.</p>"))